给出三个文件，词典库vocab.txt（任何不在词典中出现的词都认为是拼写错误），spell-errors.txt（给出正确单词与常见的拼写错误）与测试数据testdata.txt（1000个存在拼写错误的句子），利用bigram语言模型与Noisy Channel Model进行文本纠错训练。  
步骤（每一步注释标注）： 

一、读取词典库vocab.txt；  
二、读取spell-errors.txt，计算概率p(错误的单词|正确的单词)；  
三、选用nltk语料库movie_reviews，构建bigram语言模型；
四、对于词库里没有的词，取编辑距离为1和2，生成候选词集合；  
五、读取testdata.txt，找出拼错词，生成候选词，并计算每个候选词的概率；  
六、选出概率最高的候选词，作为修改词。  
注：为便于计算，可采用似然对数，将概率乘积转化为求和。注意平滑。  
输出：对每个句子，输出句子编号，拼错词，修改词（若没有则输出False），候选词列表及概率。最后输出总拼错词数。 

**例如：**   
>第 1 句：
protectionst protectionist\
[('protectionist', -30.39215205399061), ('protection', -30.39215205399061), ('protectionism', -30.39215205399061), ('projections', -30.39215205399061)]\
第 2 句：\
Tkyo's False\
第 3 句：\
retaiation retaliation\
[('retaliation', -29.442994506536245), ('relaxation', -30.829288867656135), ('reparation', -30.829288867656135), ('retardation', -30.829288867656135)] \
第 3 句：\
Japan's Japanese\
[('Japanese', -30.615365993495722), ('Japan', -30.615365993495722)]\
......


----

#### 第一步 : 读取词汇表

In [1]:
vocab = set([line.strip() for line in open('./data/hw4-data-vocab.txt')])
len(vocab)

48227

----

#### 第二步：读取spell-errors.txt，统计p(错误的单词|正确的单词)的概率分布；

In [2]:
misspell_prob = {}

for line in open('./data/hw4-data-spell-errors.txt'):
    items = line.split(':')
    correct = items[0].strip()
    misspells = [item.strip() for item in items[1].split(',')]
    lenmisspells = 0 # 错误的拼写数量
    
    misspell_prob[correct] = {} # 建立correct对mistake的对应表
    cnt = 0 #有些后缀不为字母，表示出现多次
    for i in misspells:
        pro = 1.0
        if(i[len(i) - 1] >= '0' and i[len(i) - 1] <= '9'):
            pro *= eval(i[len(i) - 1]) 
            cnt += eval(i[len(i) - 1]) 
        else:
            cnt += 1
        misspell_prob[correct][i] = pro # 每个mistake对correct的出现频率
    for i in misspells:
        misspell_prob[correct][i] /= cnt # 除以总出现次数为出现概率
    #print(misspells, len(misspells), cnt)


# print(items)
misspell_prob

{'raining': {'rainning': 0.5, 'raning': 0.5},
 'writings': {'writtings': 1.0},
 'disparagingly': {'disparingly': 1.0},
 'yellow': {'yello': 1.0},
 'four': {'forer': 0.08333333333333333,
  'fours': 0.08333333333333333,
  'fuore': 0.08333333333333333,
  'fore*5': 0.4166666666666667,
  'for*4': 0.3333333333333333},
 'woods': {'woodes': 1.0},
 'hanging': {'haing': 1.0},
 'aggression': {'agression': 1.0},
 'looking': {'loking': 0.09090909090909091,
  'begining': 0.09090909090909091,
  'luing': 0.09090909090909091,
  'look*2': 0.18181818181818182,
  'locking': 0.09090909090909091,
  'lucking': 0.09090909090909091,
  'louk': 0.09090909090909091,
  'looing': 0.09090909090909091,
  'lookin': 0.09090909090909091,
  'liking': 0.09090909090909091},
 'eligible': {'eligble': 0.3333333333333333,
  'elegable': 0.3333333333333333,
  'eligable': 0.3333333333333333},
 'electricity': {'electrisity': 0.25,
  'electricty*2': 0.5,
  'electrizity': 0.25},
 'scold': {'schold': 0.5, 'skold': 0.5},
 'adaptable':

-----

#### 第三步：选用nltk语料库movie_reviews，构建bigram语言模型；统计p(正确的单词)的概率分布。
> 可以尝试使用不同的预料库来构建bigram模型；movie_reviews中的语料库较小，test_data中很多词都在该语料库中没有包含，模型性能较差。
> 建议方法：\
1.nltk中多个语料库共同构建。\
2.自定义更相关的语料库(观察test_data中主要是什么领域的文本，找相关领域的语料集)。

In [3]:
from nltk.corpus import movie_reviews

# 读取语料库
categories = movie_reviews.categories()
corpus = movie_reviews.sents(categories=categories)

# bigram Condition on the previous word
# 根据前一个单词预测后一个单词

words = {}
bigrams = {}
start = '(START)'
end = '(END)'
word_cnt = 0.0
bi_cnt = 0.0

for sentence in corpus:
    sentence : list # 遍历语料库的句子
    sentence.insert(0, start) # 加上开始标志
    sentence.append(end)
    
    for i in range(len(sentence)):
        word = sentence[i] 
        word_cnt += 1
        # 计算单词和双词的出现频率
        # 这里没有去掉符号，后续可以像前面的作业那样去掉标点，小写化再进行后续操作
        if word in words:
            words[word] += 1 
        else:
            words[word] = 1
    
    for i in range(len(sentence) - 1):  
        bi_cnt += 1  
        bigram = sentence[i : i + 2]
        bigram = bigram[0] + " " + bigram[1]
        if bigram in bigrams:
            bigrams[bigram] += 1
        else:
            bigrams[bigram] = 1
    
for word in words:
    words[word] /= word_cnt
    # print(words[word])
for bigram in bigrams:
    bigrams[bigram] /= bi_cnt


In [4]:
words

{'(START)': 0.0380660500625315,
 'plot': 0.0008825574449816139,
 ':': 0.001774447949526814,
 'two': 0.001114717301625819,
 'teen': 8.808074963134415e-05,
 'couples': 1.5749538013551602e-05,
 'go': 0.0006492309558919605,
 'to': 0.01862937020514065,
 'a': 0.02222784798312583,
 'church': 4.0248819367965205e-05,
 'party': 0.00010674686875851642,
 ',': 0.045333586881451476,
 'drink': 1.866611912717227e-05,
 'and': 0.02075205793963377,
 'then': 0.000830642301159166,
 'drive': 6.124820338603401e-05,
 '.': 0.03842653948817502,
 '(END)': 0.0380660500625315,
 'they': 0.0028145007746439438,
 'get': 0.0011368833180893361,
 'into': 0.0015300384522054019,
 'an': 0.0033505683833274223,
 'accident': 6.066488716330988e-05,
 'one': 0.0034135665353816286,
 'of': 0.019904499468015605,
 'the': 0.04464060720885521,
 'guys': 0.00015632874769006775,
 'dies': 6.066488716330988e-05,
 'but': 0.005036352267000168,
 'his': 0.005592252627256268,
 'girlfriend': 0.00012716293655386108,
 'continues': 5.133182759972374

-----

#### 第四步：定义生成候选词集合的函数。对于词库里没有的词，取编辑距离为1和2，生成候选词集合；

In [5]:

def generate_edit_one(word : str):
    # the first list is used for generating candidates words whose distance are two from the original word
    # the second list is used to store words whose distance are one from the original word
    alphabet = ''
    for i in range(ord('a'), ord('z')):
        alphabet += chr(i)
    split = []
    for i in range(len(word) + 1):
        split.append((word[: i], word[i :]))

    # 距离为1 分别是插入 删除 替换
    insert1 = [L + letter + R for L,R in split for letter in alphabet]
    delete1 = [L + R[1 :] for L,R in split]
    replace1 = [L + letter + R[1 :] for L,R in split for letter in alphabet]

    # 去重 并选出存在于词典的词
    dis_1 = list(set(insert1 + delete1 + replace1))
    distance_1 = []
    for word1 in dis_1:
        if word1 in vocab:
            distance_1.append(word1)
    distance_1 = set(distance_1).difference(set(word))
    return dis_1, distance_1

def generate_edit_two(word : str) -> list:
    alphabet = ''
    for i in range(ord('a'), ord('z')):
        alphabet += chr(i)
    dis_1, distance_1 = generate_edit_one(word)

    # 距离为2 
    # 需要主力这里split要对dis_1处理
    # 也就是说在距离为1的所有可能上进行距离2的进一步预测
    split = []
    for j in range(len(dis_1)):
        for i in range(len(dis_1[j]) + 1):
            split.append((dis_1[j][: i], dis_1[j][i :]))

    insert2 = [L + letter + R for L,R in split for letter in alphabet]
    delete2 = [L + R[1 :] for L,R in split]
    replace2 = [L + letter + R[1 :] for L,R in split for letter in alphabet]
    dis_2 = set(insert2 + delete2 + replace2)
    distance_2 = []
    for word2 in dis_2:
        if word2 in vocab:
            distance_2.append(word2)
    #print(len(distance_1), len(distance_2))
    # 距离为2的字一定包含距离为1的字，因为replace换相同字母相当于没换
    # 所以这里不再需要 distance_2 + distance_1再去重
    distance_1 = set(distance_2).difference(set(word)) 
    # 这里去掉word本身，不去也可以 
    # 个人认为如果去掉word本身，要进行“存在词但不符合前后文”的纠错的时候
    # 可以设置一个值域来判定是否替换
    return distance_2

    # 看到了个别的写法
    # return set([e2 for e1 in generate_edit_one(word) 
    #       for e2 in generate_edit_one(e1) 
    #       if e2 in vocab])

a = generate_edit_two('wolrd')
print(len(a))

39


-----

#### 第五步：读取testdata.txt，找出拼错词，生成候选词，并计算每个候选词的概率； 选出概率最高的候选词，作为修改词。

>*为避免文件过大，输出前100句结果即可。*

In [7]:
import numpy as np
import math
import re

mistake_word_num = 0
with open('./data/hw4-data-testdata.txt') as file:
    for line in file:
        items = line.split('\t')
        n=items[0]
        word_list = items[2].split()

        # 去掉标点，避免匹配问题
        for index, word in enumerate(word_list):
            word_list[index] = re.sub("[.,]", "", word)
        


        if int(n) > 100:
            break
        for index, word in enumerate(word_list):
            #word = word.strip(',.') 前面消除了
            if word not in vocab:
                print("第",n,"句：")
                mistake_word_num += 1


                # 先生成编辑距离1的候选词，找不到候选词时再生成编辑距离2的候选词
                candidates = generate_edit_one(word) # 前面实现的时候返回元组
                candidates = candidates[1]

                if len(candidates) == 0:
                    # 同时生成编辑距离1和2的候选词
                    # 这里前面的算法实现的时候已经将 1 包含在 2 中了
                    candidates =  generate_edit_two(word) 

                if len(candidates) == 0:
                    print("没有合适的修改词")
                    continue
                # print(candidates)

                # 对于每一个candidate, 计算它的prob
                # prob = p(correct)*p(mistake|correct)
                #       = log p(correct) + log p(mistake|correct)
                # 返回prob最大的candidate
                max_pro = -1e6
                chosen = ''
                can_with_prob = {}

                for candi in candidates:
                    prob = 0.0
                    # candi 是正确词， word是错误词，应使用mispell数组
                    
                    if candi in misspell_prob and word in misspell_prob[candi] :
                        prob += math.log(misspell_prob[candi][word])
                    else:
                        #prob -= 1e4 # 没有这个词要降低权重
                        prob += math.log(1.0 / (len(misspell_prob) + len(vocab))) 
                        #减常数过于粗暴试了试平滑
                        # 不过想了下，平滑本质还是一个常数
                    
                    # 下面处理bi模型
                        
                    #print(word_list)
                    idx = word_list.index(word)
                    bi_word = " " + candi
                    pre = ""
                

                    # 前一个单词加
                    if idx > 0:
                        pre = word_list[idx - 1]
                    else:
                        pre = start # '(START)'
                    bi_word = pre + bi_word
                    # print(bigram[bi_word] + 1.0)
                    # print(words[pre] + word_cnt)

                    if bi_word in bigrams and pre in words:
                        prob += math.log( (bigrams[bi_word] + 1.0)
                        / (words[pre] + word_cnt) )
                    else:
                        #prob -= 1e4
                        prob += math.log(1.0 / (word_cnt))
                    
                    if prob > max_pro:
                        max_pro = prob
                        chosen = candi
                    can_with_prob[candi] = prob
                print("候选词集合", can_with_prob)
                print("错误词为 "+ word +" 候选词为 ", chosen, "评分为：", max_pro)

第 1 句：
候选词集合 {'protectionist': -25.28885691038303}
错误词为 protectionst 候选词为  protectionist 评分为： -25.28885691038303
第 2 句：
没有合适的修改词
第 3 句：
候选词集合 {'retaliation': -25.288855098611428}
错误词为 retaiation 候选词为  retaliation 评分为： -25.288855098611428
第 3 句：
候选词集合 {'Japan': -25.28885691038303, 'Japanese': -25.28885691038303}
错误词为 Japan's 候选词为  Japan 评分为： -25.28885691038303
第 4 句：
候选词集合 {'gases': -25.28885691038303, 'cases': -25.28885691038303, 'bases': -25.28885691038303, 'vases': -25.28885691038303, 'taxes': -25.28885691038303, 'tapes': -25.28885691038303, 'tasks': -25.28885691038303, 'takes': -25.28885630442337, 'tastes': -25.28885691038303, 'tales': -25.28885691038303, 'oases': -25.28885691038303}
错误词为 tases 候选词为  takes 评分为： -25.28885630442337
第 5 句：
没有合适的修改词
第 5 句：
候选词集合 {'business': -25.288855698236446}
错误词为 busines 候选词为  business 评分为： -25.288855698236446
第 5 句：
没有合适的修改词
第 7 句：
候选词集合 {'Taiwan': -25.28885691038303, 'Train': -25.28885691038303, 'Twain': -25.28885691038303, 'Tanin': -25.2888569103